In [ ]:
# External libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import matplotlib.style as style
import cv2
import time
import os
from utility.utils import *
import re
import h5py
import tables

# Own Libraries
from utility.utils import *
from Match.pair import *
from Match.icp import *

%matplotlib tk
style.use('seaborn-paper')

# CATALOG CREATION

In [ ]:
DB = pd.read_csv("DATA/lunar_crater_database_robbins_2018.csv", sep=",")
DB['DIAM_CIRC_IMG'] *= 0.5

def craters_from_HLL(H,lat,lon, filter = 1):
    t = find_dteta(H)  # d/R, where d^2 is the footprint
    lat_bounds, lon_bounds = (np.array([lat - t/2, lat + t/2]), np.array([lon - t/2, lon + t/2]) )
    df = CatalogSearch(DB, lat_bounds, lon_bounds, CAT_NAME="ROBBINS")
    df.rename(columns={'Diam':'r'}, inplace=True)
    df = df[ df.r > filter]        # km > filter
    df.r = df.r * 1/deg2km    #km2deg
    # Now it's all expressed in deg!
    return df

def HLL_catalog(df_mat):
    H = find_all_triplets(df_mat)
    Q = pd.DataFrame(H, columns=['Angle1','Angle2','Angle3','des1','des2','des3','x1','y1','r1','x2','y2','r2','x3','y3','r3'])
    # Retrasform radius to km:
    Q.r1 *= deg2km
    Q.r2 *= deg2km
    Q.r3 *= deg2km
    Q.rename( columns={'x1':'lon1','x2':'lon2','x3':'lon3','y1':'lat1','y2':'lat2','y3':'lat3'}, inplace=True)
    return Q

In [ ]:
lon, lat, H = -180, 3, 50
%cd DATA/CATALOG
for i in range(3):
    df = craters_from_HLL(H,lat,lon)
    df_mat = np.array(df)
    cat = HLL_catalog(df_mat)

    # cat.to_hdf(f'test_lat_{lat}_lon_{lon}.h5', key='df', format='table')
    cat.to_hdf(f'part_{i+1}.h5',key='df', format='table', index=False)
    lon+=2

In [ ]:
# LOAD ALL H5:
import glob 
dict = {}
for elem in glob.glob(f"/home/sirbastiano/Desktop/Python Projects/TRN/DATA/CATALOG/*"):
    txt = elem             # stringa completa
    name  = txt.split('/')[-1]
    t = txt.split('_')[1] # numero
    t = t.split('.')[0]
    dict[t] = name

In [ ]:
dfs=[]
for file in dict:
    h = pd.HDFStore(dict[str(file)])
    dfs.append(h['df'])

df = pd.concat(dfs)



# Merge H+L

In [ ]:
H = pd.read_csv('DATA/HeadCraters.csv')
L = pd.read_csv('DATA/LROCCraters.csv')

In [ ]:
H.rename( columns={'Lon':'lon','Lat':'lat','Diam_km':'diam' }, inplace=True)
H.head(2)

In [ ]:
L.rename(columns={'Long':'lon', 'Lat':'lat','Diameter (km)':'diam' }, inplace=True)
L.head(2)

In [ ]:
lon1 = H.lon
lon2= L.lon

lat1= H.lat
lat2=L.lat

diam1=H.diam
diam2=L.diam 

# H_L =

In [ ]:
lon1 =  np.array(lon1)
lon1 = np.vstack(lon1)
lon2 =  np.array(lon2)
lon2 = np.vstack(lon2)

lat1 =  np.array(lat1)
lat1 = np.vstack(lat1)
lat2 =  np.array(lat2)
lat2 = np.vstack(lat2)

diam1 =  np.array(diam1)
diam1 = np.vstack(diam1)
diam2 =  np.array(diam2)
diam2 = np.vstack(diam2)


LON = np.vstack([lon1,lon2]) 
LAT = np.vstack([lat1,lat2]) 
DIAM = np.vstack([diam1,diam2]) 

DATA = np.hstack([LON,LAT,DIAM])

In [ ]:
H_L = pd.DataFrame( DATA, columns=['lon','lat','diam'])
H_L

In [ ]:
H_L.to_csv('H_L_combined.csv', index=False)

# CATALOG CREATION H+L

In [ ]:
%cd DATA/CATALOG

In [ ]:
DB = pd.read_csv("/home/sirbastiano/Documenti/Python Projects/TRN/TRN/DATA/H_L_combined.csv", sep=",")

def craters_from_HLL(H,lat,lon, filter = 0):
    t = find_dteta(H)  # d/R, where d^2 is the footprint
    lat_bounds, lon_bounds = (np.array([lat - t/2, lat + t/2]), np.array([lon - t/2, lon + t/2]) )
    df = CatalogSearch(DB, lat_bounds, lon_bounds, CAT_NAME="COMBINED")
    if df is not None:
        df.rename(columns={'Diam':'r'}, inplace=True)
        df.r = df.r * 1/deg2km    #km2deg
        # Now it's all expressed in deg!
        return df
    else:return None

def HLL_catalog(df_mat):
    H = find_all_triplets(df_mat)
    Q = pd.DataFrame(H, columns=['Angle1','Angle2','Angle3','des1','des2','des3','x1','y1','r1','x2','y2','r2','x3','y3','r3'])
    # Retrasform radius to km:
    Q.r1 *= deg2km
    Q.r2 *= deg2km
    Q.r3 *= deg2km
    Q.rename( columns={'x1':'lon1','x2':'lon2','x3':'lon3','y1':'lat1','y2':'lat2','y3':'lat3'}, inplace=True)
    return Q

In [ ]:
lon, lat, H = -180, -90, 60
while lat < 90:
    lon = -180
    while lon < 180:
        df = craters_from_HLL(H,lat,lon)
        if df is not None:
            df_mat = np.array(df)
            cat = HLL_catalog(df_mat)
            cat.to_hdf(f'cat_lat_{lat}_lon_{lon}.h5', key='df', format='table')
        
        lon+=3
    lat+=3

# MERGE

In [2]:
# LOAD ALL H5:
import glob
dfs, names = [], []
for name in glob.glob(f"/home/sirbastiano/Documenti/Python Projects/TRN/TRN/DATA/CATALOG/*"):
    names.append(name)

In [4]:
for i in range(len(names)):
    printProgressBar(i,len(names), printEnd='' )
    h = pd.HDFStore(names[i])
    dfs.append(h)

 |███████████████████████████████████████████████████████████████████████████████████████████████████-| 100.0% 

In [ ]:
df = pd.concat(dfs)

In [23]:
dfs[122]['df']

,Angle1,Angle2,Angle3,des1,des2,des3,lon1,lat1,r1,lon2,lat2,r2,lon3,lat3,r3
0,5.929841,160.584196,13.485962,0.378319,1.809078,0.522025,-159.507369,5.432325,42.815601,-159.652340,4.259096,27.718500,-158.318355,7.821309,125.158675
1,0.291066,179.112111,0.596824,0.293224,1.746023,6.816749,-159.507369,5.432325,42.815601,-159.652340,4.259096,27.718500,-159.247428,7.842292,8.941095
2,7.205378,155.437962,17.356660,0.426045,1.844599,6.106951,-159.507369,5.432325,42.815601,-159.652340,4.259096,27.718500,-158.033835,7.826949,11.116723
3,9.413847,147.700329,22.885824,0.452907,1.813213,6.268449,-159.507369,5.432325,42.815601,-159.652340,4.259096,27.718500,-157.725451,7.606017,10.724232
4,11.643254,139.933438,28.423309,0.479541,1.765972,11.982643,-159.507369,5.432325,42.815601,-159.652340,4.259096,27.718500,-157.464628,7.329852,5.504388
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7975,55.856137,103.308729,20.835134,4.569596,8.220458,5.331764,-159.077698,4.148193,5.911359,-160.126928,6.625366,7.079449,-157.937907,4.341932,7.451070
7976,66.793805,91.155604,22.050592,4.933734,7.871516,3.563714,-159.077698,4.148193,5.911359,-160.126928,6.625366,7.079449,-158.057400,4.556281,9.949707
7977,83.953793,68.253530,27.792677,5.758787,7.211431,2.139190,-159.077698,4.148193,5.911359,-160.126928,6.625366,7.079449,-160.338812,4.121576,13.834544
7978,75.481734,8.850231,95.668035,9.300661,3.829196,4.053122,-159.077698,4.148193,5.911359,-160.126928,6.625366,7.079449,-159.751632,6.830185,7.237624


In [3]:
printProgressBar?

Signature:
printProgressBar(
    iteration,
    total,
    prefix='',
    suffix='',
    decimals=1,
    length=100,
    fill='█',
    printEnd='\r',
)
Docstring:
    Call in a loop to create terminal progress bar
    @params:
        iteration   - Required  : current iteration (Int)
        total       - Required  : total iterations (Int)
        prefix      - Optional  : prefix string (Str)
        suffix      - Optional  : suffix string (Str)
        decimals    - Optional  : positive number of decimals in percent complete (Int)
        length      - Optional  : character length of bar (Int)
        fill        - Optional  : bar fill character (Str)
", "
") (Str)
    
File:      ~/Documenti/Python Projects/TRN/TRN/utility/utils.py
Type:      function


In [ ]:
# # def image_gen(PATH):
# path = 'DATA/ephemeris sat/20 step size/IMG_20_Lat_0_Long_-127.683.jpg'

# # Actual Position:
# num =20
# l = -127.683
# H = 50
# X_c, Y_c = l, 0
# # x_f, y_f, z_f = spherical2cartesian(0, crater.Lat, crater.Lon)

# # Find reference in catalogs:
# dteta = find_dteta(H) # d/R, where d^2 is the footprint
# sp = dteta/2 # span
# lat_bounds, lon_bounds = np.array([Y_c-sp,Y_c+sp]), np.array([X_c-sp,X_c+sp])
# print('*'*35 +'BORDERS'+'*'*35)
# print(f'LAT: {lat_bounds}              LON:{lon_bounds}')
# print('*'*35 +'*******'+'*'*35)
# # 3 crater catalogs: HEAD | LROC | ROBBINS
# CATALOGS = ['ROBBINS', 'HEAD', 'LROC']
# for DATA in CATALOGS:
#     if DATA =='HEAD':
#         filepath = '/DATA/HeadCraters.csv'
#     elif DATA == 'LROC':
#         filepath = '/DATA/LROCCraters.csv'
#     elif DATA == 'ROBBINS':
#         filepath= 'DATA/lunar_crater_database_robbins_2018.csv'

#     DB = pd.read_csv(filepath, sep=',')
#     df = CatalogSearch(DB, lat_bounds, lon_bounds, CAT_NAME=DATA)
#     if df is None:
#         print('No Craters found!')
#     else:     
#         print(f'Found {df.shape[0]} craters!\n\n', df)
    
#     ## DRAWINGS: 
#     ## FIG.1
#     img = cv2.imread(path)
#     plt.ion()
#     plt.figure('Tile and Mask -'+DATA, dpi=200)
#     plt.subplot(1,2,1)
#     plt.imshow(img)
#     plt.subplot(1,2,2)
#     I = draw_craters(df, lon_bounds, lat_bounds)
#     plt.imshow(I)
#     plt.show()
#     plt.savefig(DATA+str(num)+'_fig-A.png')
    
#     ## FIG.2
#     image_with_craters = draw_craters_on_image(df,  lon_bounds, lat_bounds, img, u=None)
#     plt.ion()
#     plt.figure('Image with craters catalogued-'+DATA, dpi=200)
#     plt.imshow(image_with_craters)
#     plt.show()
#     plt.savefig(DATA+str(num)+'_fig-B.png')

# plt.close('all')